### Welcome to Week 5 Day 1

AutoGen AgentChat!

This should look simple and familiar, because it has a lot in common with Crew and OpenAI Agents SDK

In [1]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

### First concept: the Model

In [2]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")

In [4]:
from autogen_ext.models.ollama import OllamaChatCompletionClient
ollamamodel_client = OllamaChatCompletionClient(model="llama3.2")

### Second concept: The Message

In [3]:
from autogen_agentchat.messages import TextMessage
message = TextMessage(content="I'd like to go to London", source="user")
message

TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 9, 11, 19, 9, 23, 583968, tzinfo=datetime.timezone.utc), content="I'd like to go to London", type='TextMessage')

### Third concept: The Agent

In [19]:
from autogen_agentchat.agents import AssistantAgent

agent = AssistantAgent(
    name="airline_agent",
    model_client=model_client,
    system_message="You are a helpful assistant for an airline. You give medium size, humorous answers.",
    model_client_stream=True
)

### Put it all together with on_messages

In [20]:
from autogen_core import CancellationToken

response = await agent.on_messages([message], cancellation_token=CancellationToken())
response.chat_message.content

"Ah, London! The city of double-decker buses, fish and chips, and an entirely different type of weather than you're probably used to. Are you ready to embrace that delightful mix of sunshine, rain, and the occasional gust of wind that could put your umbrella to the test? If you're planning to visit, make sure to brush up on your British accents—or at least your tea-drinking skills! So, how can I help you bring your London dreams to life?"

### Let's make a local database of ticket prices

In [6]:
import os
import sqlite3

# Delete existing database file if it exists
if os.path.exists("tickets.db"):
    os.remove("tickets.db")

# Create the database and the table
conn = sqlite3.connect("tickets.db")
c = conn.cursor()
c.execute("CREATE TABLE cities (city_name TEXT PRIMARY KEY, round_trip_price REAL)")
conn.commit()
conn.close()

In [7]:
# Populate our database
def save_city_price(city_name, round_trip_price):
    conn = sqlite3.connect("tickets.db")
    c = conn.cursor()
    c.execute("REPLACE INTO cities (city_name, round_trip_price) VALUES (?, ?)", (city_name.lower(), round_trip_price))
    conn.commit()
    conn.close()

# Some cities!
save_city_price("London", 299)
save_city_price("Paris", 399)
save_city_price("Rome", 499)
save_city_price("Madrid", 550)
save_city_price("Barcelona", 580)
save_city_price("Berlin", 525)

In [8]:
# Method to get price for a city
def get_city_price(city_name: str) -> float | None:
    """ Get the roundtrip ticket price to travel to the city """
    conn = sqlite3.connect("tickets.db")
    c = conn.cursor()
    c.execute("SELECT round_trip_price FROM cities WHERE city_name = ?", (city_name.lower(),))
    result = c.fetchone()
    conn.close()
    return result[0] if result else None

In [9]:
get_city_price("Rome")

499.0

In [10]:
from autogen_agentchat.agents import AssistantAgent

smart_agent = AssistantAgent(
    name="smart_airline_agent",
    model_client=model_client,
    system_message="You are a helpful assistant for an airline. You give short, humorous answers, including the price of a roundtrip ticket.",
    model_client_stream=True,
    tools=[get_city_price],
    reflect_on_tool_use=True
)

In [11]:
response = await smart_agent.on_messages([message], cancellation_token=CancellationToken())
for inner_message in response.inner_messages:
    print(inner_message.content)
response.chat_message.content

[FunctionCall(id='call_mD1Zb6y6cvhVL6UbilzOKhw1', arguments='{"city_name":"London"}', name='get_city_price')]
[FunctionExecutionResult(content='299.0', name='get_city_price', call_id='call_mD1Zb6y6cvhVL6UbilzOKhw1', is_error=False)]


'Sure, you can fly to London for a roundtrip ticket priced at $299! Just remember, the only “pint” you’ll be lifting is at the pub, not in your bag! 🍻✈️'

# Full App

In [27]:
from dotenv import load_dotenv
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken

load_dotenv(override=True)

model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
message = TextMessage(content="Tell me the Weather in Dublin tomorrow", source="user")
agent = AssistantAgent(name = "Weather_checker",
                        model_client=model_client,
                        system_message="You are a weather reporter for the city of Dublin. You tell the only the average temperature for the next day by specifying the date",
                        model_client_stream=True)

response = await agent.on_messages([message], cancellation_token=CancellationToken())
response.chat_message.content

'The average temperature in Dublin tomorrow, October 19, 2023, is expected to be around 14°C.'

In [40]:
from dotenv import load_dotenv
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken

load_dotenv(override=True)

model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
agent = AssistantAgent(name = "Weather_checker",
                        model_client=model_client,
                        system_message="You are a weather reporter for the city of Dublin. You tell the only the average temperature for the next day by specifying the date",
                        model_client_stream=True)
history = []

async def Chat(user_input: str):
    global history
    message = TextMessage(content=user_input, source="user")
    history.append(message)
    response = await agent.on_messages(history, cancellation_token=CancellationToken())
    history.append(response.chat_message)

    print("Agent:", response.chat_message.content)

In [41]:
await Chat("whats the weather for tomorrow in DUblin")

Agent: The average temperature for tomorrow, October 13, 2023, in Dublin is expected to be around 14°C.


In [42]:
await Chat("How about day after")

Agent: The average temperature for October 14, 2023, in Dublin is expected to be around 15°C.
